# Agent FaaS Deployment with HydrA FaaS Manager

This notebook demonstrates function deployment using the HydrA FaaS Manager with the Agent FaaS middleware. It covers two primary methods:
1. **Inline Code**: Defining function code directly within the notebook.
2. **Project Directories**: Deploying functions from local project directories.

## Prerequisites
- The Agent FaaS middleware must be running and configured (on `localhost:8000` by default).
- Docker must be running, and you should be logged into your container registry (`docker login`).

In [1]:
import os
import shutil
import requests
from pathlib import Path
import json
import time
import sys

sys.path.insert(0, os.path.abspath('..'))

from src.hydraa_faas.faas_manager.manager import FaasManager
from hydraa.services import ServiceManager
from hydraa import Task

## 1. Initial Setup

First, we define a mock proxy to configure the `FaasManager`, check the agent's health, and initialize the manager.

In [2]:
class MockProxy:
    """A mock proxy to simulate credential loading for the FaaS Manager."""
    def __init__(self):
        self.loaded_providers = ['agent']
    
    def _load_credentials(self, provider):
        if provider == 'agent':
            return {'host': 'localhost', 'port': 8000}
        return {}

agent_url = "http://localhost:8000"

print("Checking agent faas middleware health...")
try:
    response = requests.get(f"{agent_url}/health", timeout=5)
    agent_healthy = response.json().get('status') == 'healthy'
except Exception as e:
    agent_healthy = False

if agent_healthy:
    print("✅ Agent is healthy and ready.")
    faas_mgr = FaasManager(proxy_mgr=MockProxy(), asynchronous=True, auto_terminate=True)
    try:
        service_mgr = ServiceManager([faas_mgr])
        service_mgr.start_services()
    except FileExistsError as e:
        # handle existing sandbox by cleaning up and restart
        sandbox_path = str(e).split("'")[1]
        shutil.rmtree(sandbox_path)
        service_mgr = ServiceManager([faas_mgr])
        service_mgr.start_services()
else:
    print("❌ Agent is not healthy. Please ensure the agent is running before continuing.")

Checking agent faas middleware health...
✅ Agent is healthy and ready.
starting faas manager with [1] providers
faas manager started with providers: ['agent']


## 2. Create Example Function Directories

This step creates the local directories and files for the functions we'll deploy.

In [3]:
base_dir = Path("temp_agent_functions")
if base_dir.exists():
    shutil.rmtree(base_dir)
base_dir.mkdir()

calc_dir = base_dir / "calculator"
calc_dir.mkdir()
with open(calc_dir / "main.py", 'w') as f:
    f.write('''
import json
def handler(context, event):
    a = event.get('a', 0)
    b = event.get('b', 0)
    return {'statusCode': 200, 'body': json.dumps({'sum': a + b})}
''')
print(f"Created calculator function at: {calc_dir.resolve()}")

text_dir = base_dir / "text_analyzer"
text_dir.mkdir()
with open(text_dir / "main.py", 'w') as f:
    f.write('''
import json
import inflect
p = inflect.engine()
def handler(context, event):
    text = event.get('text', '')
    count = len(text.split())
    return {'statusCode': 200, 'body': json.dumps({'message': f'The text has {p.number_to_words(count)} words.'})}
''')
with open(text_dir / "requirements.txt", 'w') as f:
    f.write('inflect')
print(f"Created text_analyzer function at: {text_dir.resolve()}")

Created calculator function at: /Users/monoble/Research/hydraa-faas/examples/temp_agent_functions/calculator
Created text_analyzer function at: /Users/monoble/Research/hydraa-faas/examples/temp_agent_functions/text_analyzer


## 3. Deploy Functions

We will now define and submit tasks for an inline function and the two functions from the directories we just created.

In [4]:
inline_task = Task()
inline_task.name = 'inline-greeter'
inline_task.provider = 'agent'
inline_task.handler_code = '''
import json
def handler(context, event):
    name = event.get('name', 'world')
    return {'statusCode': 200, 'body': json.dumps({'message': f'Hello, {name} from an inline function!'})}
'''
inline_task.handler = 'main:handler'
inline_task.runtime = 'python:3.9'

calculator_task = Task()
calculator_task.name = 'directory-calculator'
calculator_task.provider = 'agent'
calculator_task.source_directory = str(calc_dir.resolve())
calculator_task.handler = 'main:handler'
calculator_task.runtime = 'python:3.9'

text_analyzer_task = Task()
text_analyzer_task.name = 'directory-text-analyzer'
text_analyzer_task.provider = 'agent'
text_analyzer_task.source_directory = str(text_dir.resolve())
text_analyzer_task.handler = 'main:handler'
text_analyzer_task.runtime = 'python:3.9'

all_tasks = [inline_task, calculator_task, text_analyzer_task]

if agent_healthy:
    print("Submitting all functions for deployment...")
    faas_mgr.submit(all_tasks)
else:
    print("Skipping submission because agent is not healthy.")

Submitting all functions for deployment...
3 function(s) submitted for deployment


## 4. Get Deployment Results

Wait for all submitted functions to finish deploying.

In [5]:
deployed_functions = []

if agent_healthy:
    print("Waiting for all deployments to complete...")
    for task in all_tasks:
        try:
            result = task.result(timeout=600) 
            print(f"✅ SUCCESS: {task.name} deployed. Result: {result}")
            deployed_functions.append(task)
        except Exception as e:
            print(f"❌ FAILED to deploy {task.name}. Error: {e}")

Waiting for all deployments to complete...
✅ SUCCESS: inline-greeter deployed. Result: {'status': 'success', 'name': 'inline-greeter'}
✅ SUCCESS: directory-calculator deployed. Result: {'status': 'success', 'name': 'directory-calculator'}
✅ SUCCESS: directory-text-analyzer deployed. Result: {'status': 'success', 'name': 'directory-text-analyzer'}


## 5. Invoke Deployed Functions

Test the functions that were deployed successfully.

In [6]:
if not deployed_functions:
    print("No functions were deployed successfully, skipping invocation.")
else:
    print("\n--- Invoking Deployed Functions ---\n")
    # This cell must be 'async' or use await in a notebook that supports it.
    for task in deployed_functions:
        payload = {}
        if 'greeter' in task.name:
            payload = {'name': 'HydrA'}
        elif 'calculator' in task.name:
            payload = {'a': 100, 'b': 55}
        elif 'analyzer' in task.name:
            payload = {'text': 'This is a test of the text analyzer'}
        
        try:
            print(f"Invoking {task.name}...")
            # --- FIX: Use 'await' to get the result from the async task ---
            response = await faas_mgr.invoke(task.name, payload=payload, provider='agent')
            print(f"Response from {task.name}: {response['function_response']}\n")
        except Exception as e:
            print(f"Error invoking {task.name}: {e}\n")


--- Invoking Deployed Functions ---

Invoking inline-greeter...
Error invoking inline-greeter: <asyncio.locks.Event object at 0x111d7b820 [unset]> is bound to a different event loop

Invoking directory-calculator...
Error invoking directory-calculator: Server error '500 Internal Server Error' for url 'http://localhost:8000/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

Invoking directory-text-analyzer...
Error invoking directory-text-analyzer: Server error '500 Internal Server Error' for url 'http://localhost:8000/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500



## 6. Cleanup

Shut down the FaaS manager to clean up all deployed functions and remove temporary directories.

In [7]:
print("Shutting down services and cleaning up resources...")
if 'service_mgr' in locals() and service_mgr.is_active():
    service_mgr.shutdown_services()
    print("✅ FaaS Manager shut down.")

if base_dir.exists():
    shutil.rmtree(base_dir)
    print(f"✅ Removed temporary directory: {base_dir.resolve()}")

print("\nCleanup complete.")

Shutting down services and cleaning up resources...


AttributeError: 'ServiceManager' object has no attribute 'is_active'